# Quick tour with QM9

This notebook showcases a simple example of training a neural network potential on the QM9 dataset with PiNN.

In [1]:
import os, warnings
import tensorflow as tf
from glob import glob
from ase.collections import g2
from pinn.io import load_qm9, sparse_batch
from pinn import get_model, get_calc
# CPU is used for documentation generation, feel free to use your GPU!
os.environ['CUDA_VISIBLE_DEVICES'] = '' 
# We heavily use indexed slices to do sparse summations,
# which causes tensorflow to complain, 
# we believe it's safe to ignore this warning.
index_warning = 'Converting sparse IndexedSlices'
warnings.filterwarnings('ignore', index_warning)

## Getting the dataset

PiNN adapts TensorFlow's dataset API to handle different datasets.

For this and the following notebooks the QM9 dataset (https://doi.org/10.6084/m9.figshare.978904) is used.  
To follow the notebooks, download the dataset and change the directory accordingly.

The dataset will be automatically split into subsets according to the split_ratio.  
Note that to use the dataset with the estimator, the datasets should be a function, instead of a dataset object.

In [2]:
filelist = glob('/home/yunqi/datasets/QM9/dsgdb9nsd/*.xyz')
dataset = lambda: load_qm9(filelist, split={'train':8, 'test':2})
train = lambda: dataset()['train'].repeat().shuffle(1000).apply(sparse_batch(100))
test = lambda: dataset()['test'].repeat().apply(sparse_batch(100))

## Defining the model
In PiNN, models are defined at two levels: models and networks. 

- A model (model_fn) defines the target, loss and training details of the network.
- A network defines the structure of the neural network.

In this example, we will use the potential model, and the PiNet network.

The configuration of a model is stored in a nested dictionary like: 

```Python
{'model_dir': 'PiNet_QM9',
 'network': {
     'name': 'PiNet',
     'params': {
         'depth': 4,
         'rc':4.0,
         'atom_types':[1,6,7,8,9]},
 'model':{
     'name': 'potential_model',
     'params':{
         'learning_rate': 3e-4,
         ...
         }
     }
}
```

Available options of the network and model can be found in the documentation.

The easiest way to specify the network function is with its name.  
You can also provide a custom function here.

In [3]:
params = {'model_dir': '/tmp/PiNet_QM9',
          'network': {
              'name': 'PiNet',
              'params': {
                  'depth': 4,
                  'rc':4.0,
                  'atom_types':[1,6,7,8,9]
              },
          },
          'model': {
              'name': 'potential_model',
              'params': {
                  'learning_rate': 1e-3
              }
          }
}
model = get_model(params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/PiNet_QM9', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Configuring the training process
The defined model is indeed a tf.Estimator object, thus, the training can be easily controlled

In [4]:
train_spec = tf.estimator.TrainSpec(input_fn=train, max_steps=1000)
eval_spec = tf.estimator.EvalSpec(input_fn=test, steps=100)

## Train and evaluate

In [5]:
tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Skipping training since max_steps has already saved.


(None, None)

## Using the model

The trained model can be used as an ASE calculator.

In [6]:
calc = get_calc(params)
atoms = g2['C2H4']
atoms.set_calculator(calc)
atoms.get_forces(), atoms.get_potential_energy()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/PiNet_QM9', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Calling model_fn.
INFO

(array([[ 0.0000000e+00, -1.3411045e-07,  7.1963463e+00],
        [ 0.0000000e+00, -2.2947788e-06, -7.1963511e+00],
        [ 0.0000000e+00,  1.3665450e+01,  9.3304653e+00],
        [ 0.0000000e+00, -1.3665451e+01,  9.3304625e+00],
        [ 0.0000000e+00,  1.3665453e+01, -9.3304634e+00],
        [ 0.0000000e+00, -1.3665451e+01, -9.3304605e+00]], dtype=float32),
 -88.39623260498047)

## Conclusion

You have trained your first PiNN model, though the accuracy is not so satisfying
(RMSE=21 Hartree!). Also, the training speed is slow as it's limited by the IO and 
pre-processing of data.  

We will show in following notebooks that:

- Proper scaling of the energy will improve the accuracy of the model.
- The training speed can be enhanced by caching and pre-processing the data.